<div align="center">
<font color="blue"><h1>GDAL - Introduction to Raster Data</h1>
<h3> Aug 2024 @ EarthScope</h3></font></div>
Original version by Piyush Agram, revised for 2024 by Gareth Funning

In this tutorial, we will demonstrate the use of GDAL software for working geospatial data in general and with radar data products generated using the ISCE software. We will start with very basics of geospatial data representations and work through various common operations involving geospatial imagery.


## 0 Notebook setup

In [ ]:
# Always run this cell at the start of each session
import os
from osgeo import gdal, osr
from pathlib import Path
import zipfile

if 'nb_dir' not in globals():
    nb_dir = os.getcwd()

## 1 Download datasets for notebook

We will download the datasets used in this notebook. 

### Dataset 1: SRTM DEM tile in geotiff format over California from opentopography

<div class="alert alert-danger">
<b>NEW ISSUE:</b> 
Starting in January 2022, the OpenTopography servers require an OpenTopography API key for all download requests.

See https://portal.opentopography.org/ for more information on getting API key. When you have the API key, replace the `nnn` in the next command with your key.
</div>

There is also more information about this OpenTopography API key in the `UNAVCO2022/0.7_Data_Search_and_Access/Data_Access_Accounts.ipynb` notebook.

In [ ]:
!curl -X GET "https://portal.opentopography.org/API/globaldem?demtype=SRTMGL1_E&south=33&north=34&west=-119&east=-118&outputFormat=GTiff&API_Key=nnn" -H "accept: */*" --output N33W119_wgs84.tif

### Dataset 2: Greenland DEM in ERDAS IMAGINE format from Ohio State University

In [ ]:
!wget -nc  https://research.byrd.osu.edu/rsl/greenland_data/dem/dem_data/dem_pclin_icesat_polster_geoidheight.img --no-check-certificate
!wget -nc https://research.byrd.osu.edu/rsl/greenland_data/dem/dem_data/dem_pclin_icesat_polster_geoidheight.txt --no-check-certificate

### Dataset 3: USGS DEM over Bushkill, PA from PSU

In [ ]:
!wget -nc https://courseware.e-education.psu.edu/downloads/natureofgeoinfo/DEM.zip

### Dataset 4: ISCE-format data from stripmapApp.py processing of Hawaii

In [ ]:
# download and unzip some pre-processed data from an AWS bucket

def asf_unzip(output_dir, file_path):
    """
    Takes an output directory path and a file path to a zipped archive.
    If file is a valid zip, it extracts all to the output directory.
    """
    ext = file_path.suffix
    assert ext == '.zip', 'Error: file_path must be the path of a zip'

    if output_dir.is_dir() and file_path.is_file():
        print(f"Extracting: {file_path}")
        try:
            zipfile.ZipFile(file_path).extractall(output_dir)
        except zipfile.BadZipFile:
            print(f"Zipfile Error.")
        return
    
path = "./stripmap.zip"
!aws --no-sign-request s3 cp s3://asf-jupyter-data-west/unavco2021/stripmap.zip $path

path = Path(path)
asf_unzip(path.parent, path)
path.unlink()

### Dataset 5: ISCE-format DEM from Copernicus DEM
Here we use a new DEM download utility developed by Scott Staniewicz at JPL called 'sardem'. The following command will download a single 1x1 degree cell of Copernicus GLO30 DEM, convert it to ellipsoid heights, and create the ISCE2 metadata files.

See https://github.com/scottstanie/sardem to install it on another computer.

In [ ]:
!sardem --bbox -119 34 -118 35 --data COP -isce
# we did not specify the output file name so it created "elevation.dem"

## 2. GIS - Geographic Information System

GIS is a framework for organizing, managing, visualizing and analyzing geospatial data. As geophysicists, geologists or geodesists, we all use these set of tools to work with data on maps. Any data that can be plotted on a map can be classified as **geospatial** data. 

Most popular free GIS tools include:

1. Generic Mapping Tools (GMT) http://gmt.soest.hawaii.edu/
2. Geospatial Data Abstraction Library (GDAL) http://www.gdal.org/
3. QGIS https://qgis.org/
4. PROJ.4 / pyproj https://proj4.org/
5. Others like Shapely, Fiona, Caropy, Basemap etc 

In this tutorial, we will focus on the <u>Geospatial Data Abstraction Library (GDAL)</u>.

### 1.1 Types of geospatial data

Geospatial data can generally be classified into 2 types

#### Raster data

From the ARCGIS webpage:

In its simplest form, a raster consists of a matrix of cells (or pixels) organized into rows and columns (or a grid) where each cell contains a value representing information. Typical examples of raster imagery include:

1. Weather radar maps
2. Satellite imagery

Clearly, from the exercises that we have been doing with SAR data - these types of imagery fall under raster datasets. This tutorial will focus on raster data manipulation.

#### Vector data

Vector data refers to datasets that attempt to represent information using goemetric constructs like points, line and polygons. Typical examples of vector data include:

1. Highways, country boundaries etc
2. Earthquake locations from a catalog
3. Polygon shaped features like coastlines etc

With SAR/InSAR, these types of datasets tend to be used in an ancillary fashion to assist in interpretation. This type of data will not discussed in detail as such in this tutorial.

### 1.2 Why GDAL?

* Free and Open Source (https://github.com/OSGeo/gdal)
* Support for over 80+ Image formats and map projections
* Command line as well as C/C++/Python/R API
* Used extensively by worlds large geospatial data services
* Extensive test suite and active developer community
* GDAL also includes extensive support for vector datasets (Not a topic for this tutorial)

<b><font color="blue">Note:<br>
    This notebook is specifically to be used with GDAL 3.x
    </font></b>

## 2. Raster Data Model in GDAL

GDAL has a very clearly defined Data Model. For a detailed description of all the aspects of the Data Model - see http://www.gdal.org/gdal_datamodel.html. In this tutorial, we will focus on some important aspects of this model.

### a. Raster Bands

A GDAL-comptabile raster image can consist of many 2D matrices contained in it as long as

1. They are of the same size 
2. They represent information from the same points, i.e the map coordinates are the same.
3. Some data formats inherently allow the different bands to be of different datatypes, but this is not very common.

Let's look at examples of single and multi-band images using the **gdalinfo** utility.

In [ ]:
#Lets look at one downloaded SRTM tile
# This is a single band file

!gdalinfo N33W119_wgs84.tif

<div class="alert alert-danger">
<b>POTENTIAL ISSUE:</b> 
If you see `ERROR 4: 'N33W119_wgs84.tif' not recognized as a supported file format.` above, then the download from OpenTopography failed. Starting in January 2022, they require an OpenTopography API key for all download requests.

See https://portal.opentopography.org/lidarAuthorizationInfo for more information on getting API key. When you have the API key, replace the `nnn` in the download command of Section 1.1 of the notebook with your key.
</div>

If you don't manage to get the OpenTopography API key and the download of `N33W119_wgs84.tif` is not successful, you can substitute with `elevation.dem` in the rest of the notebook.

In [ ]:
#Lets look at the downloaded Copernicus DEM
# This is a single band file

!gdalinfo elevation.dem

In [ ]:
#Lets look at a coherence product generated by ISCE
# This is a two band file

!gdalinfo stripmap/interferogram/topophase.cor.geo.vrt

In [ ]:
###How do we get this information programatically
ds = gdal.Open('stripmap/interferogram/topophase.cor.geo.vrt', gdal.GA_ReadOnly)
print('Bands = ', ds.RasterCount)
print('Dimensions = ', ds.RasterXSize, ' x ', ds.RasterYSize)
ds = None

### b. Data types

The following data types are supported by GDAL:

1. Byte 
2. UInt16 
3. Int16 
4. UInt32 
5. Int32 
6. Float32
7. Float64
8. **CInt16**
9. **CInt32**
10. **CFloat32**
11. **CFloat64** 

Fact that GDAL includes support for complex numbers makes it significantly more relevant than other GIS systems for use with radar datasets.

In [ ]:
!gdalinfo stripmap/interferogram/topophase.flat.vrt

In [ ]:
##Programmatic method of querying data type
ds = gdal.Open('elevation.dem', gdal.GA_ReadOnly)
band = ds.GetRasterBand(1)
print(gdal.GetDataTypeName(band.DataType))
band = None
ds = None

### c. Coordinate System

This is where GDAL really stands out. There are numerous conventions used globally for representing the coordinate system for map data. Here are some popular ones:

1. PROJ4 - https://proj4.org/usage/projections.html
2. USGS - Collection of 18 numbers, extremely unusable
3. OGC WKT (defacto standard) - http://docs.opengeospatial.org/is/12-063r5/12-063r5.html
4. EPSG codes (Easy to use) - http://spatialreference.org/ref/epsg/
5. XML etc

We will use the **gdalsrsinfo** to demonstrate the complications involved in supporting these coordinate sytems.

In [ ]:
##Simple WGS84 lat/lon projection has EPSG code 4326.
##To look at all equivalent representations of this system
!gdalsrsinfo -o all EPSG:4326

In [ ]:
##Lets look at UTM zone 20N
!gdalsrsinfo -o all EPSG:32620

You can also use **gdalsrsinfo** to query the coordinate system of a given raster product in a specific format.

In [ ]:
##Get projection information for a raster file
!gdalsrsinfo -o proj4  N33W119_wgs84.tif

In [ ]:
## Get projection information for another raster file that has the projection information
!gdalsrsinfo -o proj4 elevation.dem

In [ ]:
###Programmatic method of querying spatial reference
ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)
srs = ds.GetProjectionRef()
ds = None
print(srs)

ref = osr.SpatialReference()
ref.ImportFromWkt(srs)
ref.ExportToProj4()

When you use **gdalsrsinfo** on a raster that does not have the coordinate system information, or as is often the case with radar products in radar geometry - it should raise an error.

In [ ]:
##This should raise an error
!gdalsrsinfo stripmap/topophase.cor

### d. Coordinates on a map

In the subsection above, we discussed coordinate systems. In this section, we talk about how GDAL associates a line/pixel location in a raster with map coordinates.

GDAL uses a simple affine system to translate (pixel, line) location to (X,Y) coordinates in map coordinates. The affine system is represented by 6 double precision numbers collectively known as the ''GeoTransform''.

**Top-left** map coordinate of a pixel represented by pixel number (P) and line number (L) in C/Python indexing is given by

$$\begin{equation*}
\left[ \begin{array}{c}
X_{left} \\
Y_{top}
\end{array} \right] = \left[ \begin{array}{c}
G_0 \\
G_3 \end{array}\right] + \left[ \begin{array}{cc}
G_1 & G_2 \\
G_4 & G_5 
\end{array} \right] \cdot \left[ \begin{array}{c}
P \\
L 
\end{array} \right]
\end{equation*}
$$


Here is the exact manner in which pixel indices are interpreted by GDAL
![Raster Layout](notebook_images/Raster_layout.png)

In [ ]:
##To confirm the Top-left interpretation, lets look gdalinfo output for the SRTM tile again
!gdalinfo N33W119_wgs84.tif

In [ ]:
###Programmatic method of querying the geotransform

ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

In [ ]:
###Programmatic method of querying the geotransform

ds = gdal.Open('elevation.dem', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

Most geospatial data formats prefer North-up orientation for storing map data. Hence, it is very common to see a negative $G_5$ term in the GeoTransform representation.

In [ ]:
###For a dataset with no coordinate system / map information - i.e, plain raster
ds = gdal.Open('stripmap/interferogram/topophase.cor.vrt', gdal.GA_ReadOnly)
print(ds.GetGeoTransform())
ds = None

### e. Ground Control Points (GCPs)

GCPs are another mechanism to associate geocoordinate information with raster images. GCPs represent an explicit mapping from a (pixel, line) location to (X,Y) location in map coordinates. These are usually not the most accurate representations, but for some applications are more than sufficient. For example, radar images acquired over very flat terrain or oceans can be geocoded reasonably well using GCPs. We are not going to use this feature in our tutorials, but one should be aware that this information is available with most SLC products delivered by different agencies.

Sentinel-1 SLC and GRD geotiff files are all annotated with GCPs. Shown below is an example output of gdalinfo on a Sentinel-1 SLC image

```bash
> gdalinfo s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff
Driver: GTiff/GeoTIFF
Files: s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff
Size is 20665, 17916
Coordinate System is `'
GCP Projection =
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["unnamed",6378137,298.2572235604902,
            AUTHORITY["EPSG","4326"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
GCP[  0]: Id=1, Info=
          (0,0) -> (130.457799786776,32.2907336141953,478.460773384615)
GCP[  1]: Id=2, Info=
          (1034,0) -> (130.506581179458,32.2986697089719,478.460773384615)
GCP[  2]: Id=3, Info=
          (2068,0) -> (130.554913356166,32.3065128120191,478.460773384615)
GCP[  3]: Id=4, Info=
          (3102,0) -> (130.602812666404,32.3142661230211,478.460773384615)
...
GCP[270]: Id=271, Info=
          (18612,17915) -> (130.826037469855,34.4318305450677,0)
GCP[271]: Id=272, Info=
          (19646,17915) -> (130.869763720912,34.438454503529,0)
GCP[272]: Id=273, Info=
          (20664,17915) -> (130.912557438728,34.4449210788505,0)
Metadata:
  AREA_OR_POINT=Area
  TIFFTAG_DATETIME=2016:04:08 16:04:16
  TIFFTAG_IMAGEDESCRIPTION=Sentinel-1A IW SLC L1
  TIFFTAG_SOFTWARE=Sentinel-1 IPF 002.62
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0,17916.0)
Upper Right (20665.0,    0.0)
Lower Right (20665.0,17916.0)
Center      (10332.5, 8958.0)
Band 1 Block=20665x1 Type=CInt16, ColorInterp=Gray
```

### f. Subdatasets

There are some file formats like netcdf or HDF5 that allow multiple datasets to be packaged together. In such cases, GDAL will report these different layers as sub-datasets. Unfortunately, the original netcdf file that we used to demonstrate this is not currently available to download, so we will just leave this up for posterity.

In [ ]:
#!gdalinfo Greenland1km.nc

To access a specific sub-dataset within the netcdf file, you can use the "SUBDATASET\_\*\_NAME" atrtibute reported by gdalinfo.

In [ ]:
#!gdalinfo NETCDF:"Greenland1km.nc":topg

In [ ]:
#!gdalsrsinfo NETCDF:"Greenland1km.nc":topg

### g. Other features

There are other useful features that GDAL provides for working with raster data. One can read more about these here: 
http://www.gdal.org/gdal_datamodel.html

## 3. Raster data formats

We have already mentioned that GDAL can support over 80+ data formats. GDAL tries to provide a unified interface to reading data from these different formats. Support for different formats depends on what libraries were used when building the GDAL library. You can find out the various raster formats that GDAL supports using **gdalinfo**.

In [ ]:
#List all supported formats in your GDAL build
!gdalinfo --formats

The capability for each format is indicated with a string in the output and here is the corresponding legend:

1. r: read
2. ro: read only
3. w: write "Create dataset by copying another"
4. w+: write with support to update "create writable dataset"
5. s: supports subdatasets
6. v: supports virtual IO - eg. /vsimem/

Virtual features are special and we will revisit this later in this tutorial.

To demonstrate that different formats can be used to represent the same data, we have included a folder called **DEM** that contains the SRTM 90m DEM for the same tile in 3 different formats - SRTM native hgt format (from LPDAAC), GMT format (from GMTSAR's dem service) and Geotiff format (OpenTopography). You can verify that the contents are same by running the command.

In [ ]:
!gdalinfo -stats N33W119_wgs84.tif

<br>
<div class="alert alert-info">
<b>Note :</b>
As you can see that GDAL supports a wide range of data formats and provides a unified interface. If you plan to work with raster datasets, we strongly recommend that you use one of these supported formats as a basis. There needs to be a really compelling reason to pick a different format.
</div>



## 4. Virtual File Systems

GDAL does not necessarily your need data to be stored on a local file. It provides a file like interface to various data sources and these are referred to as Virtual Formats. Such systems are called Virtual File Systems. For a complete list of Virtual File Systems supported by GDAL - see http://www.gdal.org/gdal_virtual_file_systems.html

We will look at a couple of examples of how these work. 

### a. vsicurl

This is meant to be used with datasets that are served online via http / https/ ftp . To enable this feature, GDAL needs to be built with "curl" support. Here is an example of how we would use this feature.
Sorry. Amazon changed their Landsat archive to require an AWS account, so this example no longer works.

In [ ]:
#This is an example of working with Amazon's Landsat archive
#!gdalinfo /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/139/045/LC08_L1TP_139045_20170304_20170316_01_T1/index.html    

### b. vsizip / vsitar / vsigzip

In [ ]:
#Look at the DEM downloaded from the PSU site
!zipinfo DEM.zip

In [ ]:
###Directly interact with data inside zip / tar files
!gdalinfo /vsizip/DEM.zip/DEM_30m/bushkill_pa.dem

### c. vsis3/ vsigs/ vsiaz

Lot of the new updates in GDAL are focused on extending support to data stored on cloud platforms - allowing users to interact with small portions of data without having to download everything.

#### Option 1: Using vsicurl and cookies with .netrc file

In this case, we will reuse the .netrc file settings that we had setup earlier for automated DEM downloads from LP DAAC. If you haven't done this, complete Step 2 from the link shown here: 
https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget

In [ ]:
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','asf_cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', 'asf_cookies.txt')
results = gdal.Info("/vsicurl/https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-043-tops-20200128_20200122-151726-00038E_00038N-PP-149c-v2_0_5.nc")
print(results)

More about how to interact with data that ASF is hosting on the Amazon Cloud in later tutorials ... 

### d. vsimem

These are feature that allows one to use in memory arrays just like other GDAL datasets. Here is an example:

In [ ]:
##Get memory driver
memdrv = gdal.GetDriverByName('MEM')

##Open source dataset in read only mode
ds = gdal.Open('N33W119_wgs84.tif', gdal.GA_ReadOnly)

##Create an in-memory copy
memds = memdrv.CreateCopy('dummyname', ds)

##Close source
ds = None

##Use copy for operations
print(memds.GetProjection())
print(memds.GetGeoTransform())

##Close memory copy
memds = None

## 5. Virtual Raster Table (VRT) format

Virtual Raster Table (VRT) format extends the idea of virtualization to actual disks on file. VRT is a custom format supported by GDAL that is similar to "links" or "shortcuts" on operating systems - except that it has more functionality than just pointing to other data sources.

Often, when working with raster datasets, we tend to generate a fairly large volume of intermediate products due to operations like cropping, multilooking and masking. VRT's offer the option of building workflow pipelines that generate such products on the fly. We will rely extensively on this feature for our time-series exercises later. 

Details on GDAL's VRT feature can be found here: http://www.gdal.org/gdal_vrttut.html

You may have already noticed that ISCE generates a **.vrt** file for every raster product that is generated by the workflows. We will be looking at VRTs in greater detail as part of the other notebooks on raster data manipulation. But here are some short examples that demonstrate the capabilities of this format.

### a. Cropping of datasets

ISCE uses GDAL VRT files to virtually extract a single burst from the Sentinel-1 SAFE products. From our TOPS processing tutorial, you will remember that Sentinel-1 TOPS mode data is delivered as SAFE products which contains TIFF files for each imaged swath. Each swath consists of number of bursts or scans. 

To avoid replicating this data on disk again, we use a simple VRT file to point to the location of burst data within the TIFF files. For example, in the topsApp.ipynb processing output, there will be processed burst SLCs arranged in folders, where "reference" is the reference image of an interferogram and "secondary" is the image to be interfered with the reference.  If you were to look at the contents of reference/IW1/burst04.slc.vrt, you will see something like 

```xml
<VRTDataset rasterXSize="20665" rasterYSize="1493">
    <VRTRasterBand dataType="CFloat32" band="1">
        <NoDataValue>0.0</NoDataValue>
        <SimpleSource>
            <SourceFilename relativeToVRT="1">/vsizip//Users/agram/data/isce/japan/data/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.SAFE/measurement/s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff</SourceFilename>
            <SourceBand>1</SourceBand>
            <SourceProperties RasterXSize="20665" RasterYSize="17916" DataType="CInt16"/>
            <SrcRect xOff="130" yOff="4498" xSize="20468" ySize="1458"/>
            <DstRect xOff="130" yOff="19" xSize="20468" ySize="1458"/>
        </SimpleSource>
    </VRTRasterBand>
</VRTDataset>
```

For clarity, lets annotate the vrt file to describe what is happening. Note that VRT format is built on top of XML. So, we will use XML's comment feature to annotate the XML for your understanding.

```xml
<!--Size of the virtual dataset-->
<VRTDataset rasterXSize="20665" rasterYSize="1493">
    <!-- Band 1 of the dataset and its data type -->
    <VRTRasterBand dataType="CFloat32" band="1">
        <!--No data value for this dataset-->
        <NoDataValue>0.0</NoDataValue>
        <!--We use a simple source to suggest that we don't alter contents of the data in source-->
        <SimpleSource>
            <!--Name of the source - notice that we point to a TIFF inside a zip file -->
            <SourceFilename relativeToVRT="1">/vsizip//Users/agram/data/isce/japan/data/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip/S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.SAFE/measurement/s1a-iw1-slc-vv-20160408t091355-20160408t091428-010728-01001f-001.tiff</SourceFilename>
            <!--Source may contain multiple bands. Indicate band from source -->
            <SourceBand>1</SourceBand>
            <!--Properties of the source. This is optional. GDAL will figure it out on the fly if not provided, but will be slow as it actually has to read the source to figure this out. Note that source is of type CInt16 -->
            <SourceProperties RasterXSize="20665" RasterYSize="17916" DataType="CInt16"/>
            <!--Indicate spatial subset of source data in pixels and lines -->
            <SrcRect xOff="130" yOff="4498" xSize="20468" ySize="1458"/>
            <!--Indicate spatial location of destination-->
            <DstRect xOff="130" yOff="19" xSize="20468" ySize="1458"/>
        </SimpleSource>
    </VRTRasterBand>
</VRTDataset>
```

### b. Modify some metadata

**Unfortunately, GMT grd support was dropped in GDAL 3.5, so we cannot use this example anymore. But just in case you find an example where coordinate system metadata are missing from geospatial data, this is one way you can correct it.**

If you have a data file that is tagged with spatial coordinate information but not with coordinate system metadata, such as a GMT grd file, some features of gdal will not work. We can fix that quickly by writing a VRT that points to the data file, and has the coordinate system information included in it.

In [ ]:
#!gdal_translate -of VRT -a_srs EPSG:4326 18.grd UCSB_18arcsec.dem.vrt

In [ ]:
#!cat UCSB_18arcsec.dem.vrt

In [ ]:
#!gdalinfo UCSB_18arcsec.dem.vrt

### c. Derived datasets

Often, we end up in situations where we need to manipulate the data slightly for use with other analysis tools. For example, derive power in dB or amplitude or phase from coregistered SLCs. Generating these derived products could use up a lot of disk space. VRTs provide a mechanism called **pixel functions** to do these transformations on the fly. Set of pixel functions available are (current list):

* "real": extract real part from a single raster band (just a copy if the input is non-complex)
* "imag": extract imaginary part from a single raster band (0 for non-complex)
* "complex": make a complex band merging two bands used as real and imag values
* "mod": extract module from a single raster band (real or complex)
* "phase": extract phase from a single raster band [-PI,PI] (0 or PI for non-complex)
* "conj": computes the complex conjugate of a single raster band (just a copy if the input is non-complex)
* "sum": sum 2 or more raster bands
* "diff": computes the difference between 2 raster bands (b1 - b2)
* "mul": multiply 2 or more raster bands
* "cmul": multiply the first band for the complex conjugate of the second
* "inv": inverse (1./x). Note: no check is performed on zero division
* "intensity": computes the intensity Re(x*conj(x)) of a single raster band (real or complex)
* "sqrt": perform the square root of a single raster band (real only)
* "log10": compute the logarithm (base 10) of the abs of a single raster band (real or complex): log10( abs( x ) )
* "dB": perform conversion to dB of the abs of a single raster band (real or complex): 20. * log10( abs( x ) )
* "dB2amp": perform scale conversion from logarithmic to linear (amplitude) (i.e. 10 ^ ( x / 20 ) ) of a single raster band (real only)
* "dB2pow": perform scale conversion from logarithmic to linear (power) (i.e. 10 ^ ( x / 10 ) ) of a single raster band (real only)


Note that this list is very relevant for SAR/InSAR users ;)